In [16]:
source = """
from my_module import thingy
from pprint import pprint

GLOBAL = "asd"

x = GLOBAL

def do_thingy(*args, do_quickly = True):
    return thingy(*args, do_quickly)

def foo(p):
    a = 1
    print(a)
    x = 2
    print(x)
    p.x = x

def bar(*args, **kwargs):
    print("before do_thingy!")
    result = do_thingy(*args, **kwargs)
    print("after do_thingy!")
    return result

if __name__ == "__main__":
    my_obj = {"x": "hi, mom!"}
    foo(my_obj)
    hello_world = "hello world" 
    # cursed thing
    hello_world, pprint = pprint, hello_world
    hello_world(pprint)

"""

In [17]:
import ast


class NameVisitor(ast.NodeVisitor):

    def get_names(self, node: ast.AST, id: str = None) -> list[ast.Name]:
        self._id = id
        self._names = []
        self.visit(node)
        return self._names


    def visit_Name(self, node: ast.Name):
        match self._id:
            case None:
                self._names.append(node)
            case id:
                if id != node.id: return
                self._names.append(node)


class SingleAssignVisitor(ast.NodeVisitor):
    
    def get_names(self, node: ast.AST) -> list[ast.Name]:
        self._names = []
        self.visit(node)
        return self._names

    def visit_Assign(self, node: ast.Assign):
        match node:
            case ast.Assign(targets=[ast.Name(ctx=ast.Store()) as name]):
                self._names.append(name)


def get_stored_ids(node: ast.AST):
    names = NameVisitor().get_names(node)
    ctxs = {}
    for name in names:
        ctxs.setdefault(name.id, name.ctx)
    return [ key for key in ctxs if isinstance(ctxs[key], ast.Store)]


def get_stored_ids_strict(node: ast.AST):  # only names of single assingment targets
    names = SingleAssignVisitor().get_names(node)
    ctxs = {}
    for name in names:
        ctxs.setdefault(name.id, name.ctx)
    return [ key for key in ctxs if isinstance(ctxs[key], ast.Store)]


class VariableRenamer(ast.NodeTransformer):

    def rename(self, node: ast.AST):
        self._stored_ids = get_stored_ids_strict(node)
        self.visit(node)
    

    def visit_Name(self, node: ast.Name):
        if (node.id in self._stored_ids):
            node.id = f"new_{node.id}"  # TODO: check if the name is available
            return node
        return node


tree = ast.parse(source)

In [18]:
print(get_stored_ids_strict(tree))

['GLOBAL', 'x', 'a', 'result', 'my_obj', 'hello_world']


In [19]:
VariableRenamer().rename(tree)
print(ast.unparse(tree))

from my_module import thingy
from pprint import pprint
new_GLOBAL = 'asd'
new_x = new_GLOBAL

def do_thingy(*args, do_quickly=True):
    return thingy(*args, do_quickly)

def foo(p):
    new_a = 1
    print(new_a)
    new_x = 2
    print(new_x)
    p.x = new_x

def bar(*args, **kwargs):
    print('before do_thingy!')
    new_result = do_thingy(*args, **kwargs)
    print('after do_thingy!')
    return new_result
if __name__ == '__main__':
    new_my_obj = {'x': 'hi, mom!'}
    foo(new_my_obj)
    new_hello_world = 'hello world'
    new_hello_world, pprint = (pprint, new_hello_world)
    new_hello_world(pprint)


In [20]:
print(ast.dump(ast.parse("my_obj.my_attr")))  # NOTE: attributes are not a name nodes!
# TODO: things needed for `VariableRenamer` implementation with `get_stored_ids` instead of `get_stored_ids_strict`
print(ast.dump(ast.parse("from pprint import pprint as pp")))  # TODO: exclude used and unused import names from `get_stored_ids`
print(ast.dump(ast.parse("x = y")))
print(ast.dump(ast.parse("x, y = y, x")))  # TODO: ignore multiple assigns like this

Module(body=[Expr(value=Attribute(value=Name(id='my_obj', ctx=Load()), attr='my_attr', ctx=Load()))], type_ignores=[])
Module(body=[ImportFrom(module='pprint', names=[alias(name='pprint', asname='pp')], level=0)], type_ignores=[])
Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Name(id='y', ctx=Load()))], type_ignores=[])
Module(body=[Assign(targets=[Tuple(elts=[Name(id='x', ctx=Store()), Name(id='y', ctx=Store())], ctx=Store())], value=Tuple(elts=[Name(id='y', ctx=Load()), Name(id='x', ctx=Load())], ctx=Load()))], type_ignores=[])
